<a href="https://colab.research.google.com/github/Duaa-Raed/bookz/blob/main/bookz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files

# 🔹 تحميل أول ملف
print("📁 حمّلي الملف الأول (مثلاً books.csv):")
uploaded1 = files.upload()
df = pd.read_csv(list(uploaded1.keys())[0])
print("✅ تم تحميل:", list(uploaded1.keys())[0])
print(df.columns)
print(df.head())




In [ ]:
# عرض أول 5 صفوف
print("👀 نظرة عامة على البيانات:")
print(df.head())

# معلومات سريعة عن الأعمدة
print("\n📊 معلومات الأعمدة:")
print(df.info())

# إحصاءات رقمية
print("\n📈 إحصاءات عامة:")
print(df.describe())

# فحص القيم الفارغة
print("\n❌ القيم المفقودة:")
print(df.isnull().sum())

In [ ]:
# 📊 التحليل البصري للبيانات (EDA)
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# ضبط إعدادات الخط العربي
plt.rcParams['font.family'] = 'Arial'

# 1️⃣ أكثر المؤلفين إنتاجًا
top_authors = df['Author'].value_counts().head(10)
plt.figure(figsize=(10,5))
sns.barplot(x=top_authors.values, y=top_authors.index, palette="viridis")
plt.title("📚 أكثر المؤلفين إنتاجًا")
plt.xlabel("عدد الكتب")
plt.ylabel("الكاتب")
plt.show()

# 2️⃣ أكثر التصنيفات انتشارًا
top_categories = df['Category'].value_counts().head(10)
plt.figure(figsize=(10,5))
sns.barplot(x=top_categories.values, y=top_categories.index, palette="magma")
plt.title("🏷️ أكثر التصنيفات انتشارًا")
plt.xlabel("عدد الكتب")
plt.ylabel("التصنيف")
plt.show()

# 3️⃣ تحليل الأسعار وعدد الصفحات
print("💰 إحصاءات الأسعار:")
print(df['Price'].describe())

print("\n📖 إحصاءات عدد الصفحات:")
print(df['Pages'].describe())

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
sns.histplot(df['Price'], bins=40, kde=True, color='orange')
plt.title("توزيع الأسعار")

plt.subplot(1,2,2)
sns.histplot(df['Pages'], bins=40, kde=True, color='blue')
plt.title("توزيع عدد الصفحات")

plt.show()

# 4️⃣ العلاقة بين السعر وعدد الصفحات
plt.figure(figsize=(7,6))
sns.scatterplot(data=df, x="Pages", y="Price", alpha=0.6)
plt.title("📈 العلاقة بين السعر وعدد الصفحات")
plt.xlabel("عدد الصفحات")
plt.ylabel("السعر (دينار)")
plt.show()


In [ ]:
# حذف الصفوف اللي فيها صفحات أو أسعار صفر
df_clean = df[(df['Pages'] > 0) & (df['Price'] > 0)]

print("📚 عدد الكتب بعد التنظيف:", len(df_clean))


df_clean = df_clean[(df_clean['Price'] < 500) & (df_clean['Pages'] < 2000)]

print("✅ بعد إزالة القيم الشاذة:", len(df_clean))

print("💰 الأسعار بعد التنظيف:")
print(df_clean['Price'].describe())

print("\n📖 الصفحات بعد التنظيف:")
print(df_clean['Pages'].describe())

In [ ]:
# تنظيف النصوص من القيم الفارغة
df.fillna("", inplace=True)

# إنشاء عمود موحّد للنصوص
df["text"] = (
    "📖 العنوان: " + df["Title"].astype(str) +
    " ✍️ المؤلف: " + df["Author"].astype(str) +
    " 🏷️ التصنيف: " + df["Category"].astype(str) +
    " 📝 الوصف: " + df["Description"].astype(str)
)

print("✅ تم تجهيز النصوص:", len(df))
print(df["text"].head(3))

In [ ]:
import pandas as pd
from google.colab import files

# 🔹 تحميل ملف البيانات (اختاري الملف من جهازك)
print("📁 حمّلي ملف الكتب (مثلاً jamalon dataset.csv):")
uploaded = files.upload()

# قراءة الملف
df1 = pd.read_csv(list(uploaded.keys())[0])
print("✅ تم تحميل:", list(uploaded.keys())[0])
print(df1.columns)

# 🔹 إنشاء عمود نصي موحد للعنوان والوصف
if "Title" in df1.columns and "Description" in df1.columns:
    df1["text"] = df1["Title"].fillna('') + " - " + df1["Description"].fillna('')
    print("✅ تم إنشاء العمود text بنجاح!")
else:
    print("⚠️ تأكدي إنو عندك أعمدة Title و Description في الملف.")


In [ ]:
!pip install faiss-cpu


In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# تحميل الموديل
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# إنشاء الـ embeddings
embeddings = model.encode(df1["text"].tolist(), show_progress_bar=True)

# تحويلها إلى float32 لـ faiss
embeddings = np.array(embeddings).astype("float32")

# بناء الفهرس للبحث
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

print(f"✅ تم إنشاء قاعدة المعرفة بنجاح! عدد الكتب: {len(embeddings)}")

In [ ]:
# إنشاء الفهرس
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)
print("✅ تم بناء قاعدة البحث:", index.ntotal)


In [ ]:
def ask(query, k=3):
    query_emb = model.encode([query]).astype("float32")
    distances, indices = index.search(query_emb, k)
    results = df.iloc[indices[0]]
    for i, row in results.iterrows():
        print(f"📚 {row['Title']} — {row['Author']}")
        print(f"📝 {row['Description'][:250]}...")
        print("—" * 60)


In [ ]:
ask("كتب عن التنمية البشرية والنجاح")

In [ ]:
ask("أفضل روايات نجيب محفوظ")

In [ ]:
from transformers import pipeline

# نستخدم موديل خفيف يدعم العربية والإنجليزية
generator = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct-v0.2")

def rag_answer(query, k=3):
    # البحث عن الكتب الأقرب
    query_emb = model.encode([query]).astype("float32")
    distances, indices = index.search(query_emb, k)
    results = df1.iloc[indices[0]]

    # تجميع النصوص من الكتب
    context = "\n\n".join(
        [f"📚 {r['Title']} — {r['Author']}\n{r['Description']}" for _, r in results.iterrows()]
    )

    # نص الإدخال للموديل
    prompt = f"""
    سؤال المستخدم: {query}

    النصوص المأخوذة من الكتب:
    {context}

    ✍️ أجب بإيجاز وبأسلوب طبيعي بالاعتماد فقط على النصوص أعلاه.
    """

    # توليد الإجابة
    answer = generator(prompt, max_new_tokens=200)[0]['generated_text']
    print("🤖 الإجابة:\n", answer)


In [ ]:
def rag_answer(question, retriever, model, tokenizer):
    # 🔍 الخطوة 1: نجيب القطع (المصادر) الأقرب للسؤال
    docs = retriever.get_relevant_documents(question)
    context = "\n".join([d.page_content for d in docs])

    # ✨ الخطوة 2: نحضر البرومبت الكامل للسؤال + السياق
    prompt = f"سؤال: {question}\n\nالمعلومات المتاحة:\n{context}\n\nالإجابة المفصلة:"

    # 🧠 الخطوة 3: نخلي النموذج يجاوب بناءً على السياق
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=300)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return answer


In [ ]:
!pip install -U langchain langchain-community faiss-cpu sentence-transformers

In [ ]:
import pandas as pd
from google.colab import files

# تحميل ملف الكتب
print("📁 حمّلي ملف الكتب (مثلاً jamalon dataset.csv):")
uploaded = files.upload()

# قراءة الملف
df = pd.read_csv(list(uploaded.keys())[0])
print("✅ تم تحميل:", list(uploaded.keys())[0])
print(df.columns)

# إنشاء عمود نصي موحّد
if "Title" in df.columns and "Description" in df.columns:
    df["text"] = df["Title"].fillna('') + " - " + df["Description"].fillna('')
    print("✅ تم إنشاء العمود text بنجاح!")
else:
    print("⚠️ تأكدي إن الأعمدة Title و Description موجودة في الملف.")


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

documents = [Document(page_content=row["text"]) for _, row in df.iterrows()]
vectorstore = FAISS.from_documents(documents, embeddings)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

print("✅ تم بناء قاعدة البحث بنجاح!")




In [ ]:
!pip install -U langchain langchain-community faiss-cpu sentence-transformers

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import pandas as pd
from google.colab import files

# 🧩 تحميل ملف الكتب
print("📁 حمّلي ملف الكتب (مثلاً jamalon dataset.csv):")
uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0])
print("✅ تم تحميل:", list(uploaded.keys())[0])

# إنشاء عمود موحد للنصوص
df.fillna("", inplace=True)
df["text"] = (
    "📖 العنوان: " + df["Title"].astype(str) +
    " ✍️ المؤلف: " + df["Author"].astype(str) +
    " 🏷️ التصنيف: " + df["Category"].astype(str) +
    " 📝 الوصف: " + df["Description"].astype(str)
)

# 🧠 إنشاء المستندات الشعاعية
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

documents = [Document(page_content=row["text"]) for _, row in df.iterrows()]
vectorstore = FAISS.from_documents(documents, embeddings)

# إنشاء retriever (أداة البحث)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})

print("✅ تم بناء قاعدة البحث بنجاح!")


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# 🔹 تحميل موديل التوليد
model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")



In [ ]:
def ask_books_bot(query):
    # 🔍 استرجاع النتائج من قاعدة البيانات
    results = retriever.invoke(query)

    # فلترة النتائج لتشمل فقط النصوص التي تحتوي على اسم الكتاب المطلوب
    filtered = [doc for doc in results if any(word in doc.page_content for word in query.split())]

    if not filtered:
        print("🤖 نعتذر، لا تتوفر معلومات عن هذا الكتاب.")
        return

    # دمج النصوص التي تم العثور عليها
    context = "\n\n".join([doc.page_content for doc in filtered])

    # 🧠 إعداد الـ prompt
    prompt = f"""أنت مساعد ذكي مختص بالكتب والروايات.
اعتمد فقط على المعلومات أدناه للإجابة بشكل مختصر وواضح دون إعادة عرضها.

المعرفة:
{context}

سؤال المستخدم: {query}
الإجابة:"""

    # 🗣️ توليد الإجابة
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=180)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # تنظيف الإخراج
    if "الإجابة:" in answer:
        answer = answer.split("الإجابة:")[-1].strip()

    # التحقق من الجودة
    if not answer or len(answer) < 15:
        print("🤖 نعتذر، لا تتوفر معلومات عن هذا الكتاب.")
    else:
        print("🤖", answer)


In [ ]:
# ==========================================
# 📚 الكود النهائي المصحح (باستخدام gemini-pro مجدداً للتأكد من التحديث)
# ==========================================
!pip install -q sentence-transformers faiss-cpu pandas google-generativeai

import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from google.colab import files
import google.generativeai as genai
import torch

# 1️⃣ إعداد Gemini API
print("🔑 احصلي على API Key مجاني من: https://makersuite.google.com/app/apikey")
api_key = input("أدخلي API Key: ").strip()
genai.configure(api_key=api_key)

# 🚨 التعديل: استخدام اسم النموذج المستقر gemini-pro
gemini_model = genai.GenerativeModel('gemini-pro')
print("✅ تم تفعيل Gemini!")

# 2️⃣ تحميل البيانات (نستخدم الملف الأخير الذي تم حفظه)
print("\n📁 ارفعي ملف الكتب:")
uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0])

df.fillna("", inplace=True)
df["text"] = (
    df["Title"].astype(str) + " | " +
    df["Author"].astype(str) + " | " +
    df["Description"].astype(str)
)

# 3️⃣ إنشاء Embeddings
print("🔍 إنشاء قاعدة البحث...")

# تنظيف ذاكرة GPU
if 'embeddings' in locals() or 'embeddings' in globals():
    del embeddings
if 'model' in locals() or 'model' in globals():
    del model
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("✅ تم تنظيف ذاكرة GPU.")

# استخدام النموذج الصحيح all-MiniLM-L6-v2
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# الحفاظ على حجم دفعة صغير
BATCH_SIZE = 16
embeddings = model.encode(df["text"].tolist(), show_progress_bar=True, batch_size=BATCH_SIZE)

embeddings_array = np.array(embeddings).astype('float32')
index = faiss.IndexFlatL2(embeddings_array.shape[1])
index.add(embeddings_array)
print(f"✅ جاهز! ({len(df)} كتاب)")

# 4️⃣ دالة البحث + Gemini
def ask_gemini(query, k=3):
    """بحث ذكي مع إجابات من Gemini"""

    # البحث في قاعدة البيانات
    query_emb = model.encode([query]).astype('float32')
    distances, indices = index.search(query_emb, k)

    # تجميع معلومات الكتب
    books_info = []
    for idx in indices[0]:
        book = df.iloc[idx]
        books_info.append(f"""
📖 {book['Title']}
✍️ المؤلف: {book['Author']}
📝 الوصف: {book['Description'][:300]}
""")

    context = "\n\n".join(books_info)

    # إعداد Prompt لـ Gemini
    prompt = f"""أنت مساعد مكتبة ذكي. استخدم المعلومات التالية للإجابة:

{context}

سؤال المستخدم: {query}

قدم إجابة مفيدة ومختصرة (3-4 جمل فقط)، واذكر أسماء الكتب المناسبة."""

    try:
        # الحصول على الإجابة من Gemini
        response = gemini_model.generate_content(prompt)

        print(f"\n🔍 سؤالك: {query}")
        print("=" * 60)
        print(f"\n🤖 الإجابة:")
        print(response.text)
        print("\n📚 الكتب المقترحة:")
        print("-" * 60)

        for i, idx in enumerate(indices[0]):
            book = df.iloc[idx]
            print(f"\n{i+1}. {book['Title']} - {book['Author']}")

    except Exception as e:
        print(f"❌ خطأ في Gemini: {e}")
        print("📚 عرض النتائج مباشرة:")
        for i, idx in enumerate(indices[0]):
            book = df.iloc[idx]
            print(f"\n{i+1}. 📖 {book['Title']}")
            print(f"   ✍️ {book['Author']}")
            print(f"   📝 {book['Description'][:200]}...")

# ==========================================
# 🧪 تجربة
# ==========================================
print("\n\n✅ النظام جاهز!")

# ... (اختبارات أسئلة الكتب)

In [ ]:
# ==========================================
# 🧪 تجربة
# ==========================================
print("\n\n✅ النظام جاهز!")

ask_gemini("أريد كتب عن الفلسفة الإسلامية")
ask_gemini("ما هي أفضل رواية عربية حديثة؟")
ask_gemini("كتب مناسبة للأطفال عمر 10 سنوات")

# وضع تفاعلي
print("\n\n💬 اسألني أي سؤال (أو 'خروج' للإنهاء):")
while True:
    q = input("\n❓ سؤالك: ").strip()
    if q.lower() in ['خروج', 'exit']:
        break
    if q:
        ask_gemini(q)

In [ ]:
# ==========================================
# 📚 الكود النهائي (نسخة العرض النظيف والمبسط)
# ==========================================
!pip install -q sentence-transformers faiss-cpu pandas google-generativeai

import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from google.colab import files
import google.generativeai as genai
import torch

# 1️⃣ إعداد Gemini API
print("🔑 احصلي على API Key مجاني من: https://makersuite.google.com/app/apikey")
api_key = input("أدخلي API Key: ").strip()
genai.configure(api_key=api_key)

# 🚨 النموذج الأحدث والأكثر توافقاً
gemini_model = genai.GenerativeModel('gemini-2.5-flash')
print("✅ تم تفعيل Gemini!")

# 2️⃣ تحميل البيانات
print("\n📁 ارفعي ملف الكتب:")
# تأكدي من إعادة رفع الملف أو أن الملف موجود في بيئة التشغيل
uploaded = files.upload()
df = pd.read_csv(list(uploaded.keys())[0])

df.fillna("", inplace=True)
df["text"] = (
    df["Title"].astype(str) + " | " +
    df["Author"].astype(str) + " | " +
    df["Description"].astype(str)
)

# 3️⃣ إنشاء Embeddings
print("🔍 إنشاء قاعدة البحث...")

# تنظيف ذاكرة GPU
if 'embeddings' in locals() or 'embeddings' in globals():
    del embeddings
if 'model' in locals() or 'model' in globals():
    del model
if torch.cuda.is_available():
    torch.cuda.empty_cache()
print("✅ تم تنظيف ذاكرة GPU.")

# استخدام النموذج الصحيح all-MiniLM-L6-v2
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# الحفاظ على حجم دفعة صغير
BATCH_SIZE = 16
embeddings = model.encode(df["text"].tolist(), show_progress_bar=True, batch_size=BATCH_SIZE)

embeddings_array = np.array(embeddings).astype('float32')
index = faiss.IndexFlatL2(embeddings_array.shape[1])
index.add(embeddings_array)
print(f"✅ جاهز! ({len(df)} كتاب)")

# 4️⃣ دالة البحث + Gemini (التعديل النهائي للمخرجات)
def ask_gemini(query, k=3):
    """بحث ذكي مع إجابات من Gemini"""

    # البحث في قاعدة البيانات وتجميع السياق (بدون تغيير)
    query_emb = model.encode([query]).astype('float32')
    distances, indices = index.search(query_emb, k)

    books_info = []
    for idx in indices[0]:
        book = df.iloc[idx]
        books_info.append(f"""
📖 {book['Title']}
✍️ المؤلف: {book['Author']}
📝 الوصف: {book['Description'][:300]}
""")
    context = "\n\n".join(books_info)

    # إعداد Prompt لـ Gemini (بدون تغيير)
    prompt = f"""أنت مساعد مكتبة ذكي. استخدم المعلومات التالية للإجابة:
{context}
سؤال المستخدم: {query}
قدم إجابة مفيدة ومختصرة (3-4 جمل فقط)، واذكر أسماء الكتب المناسبة."""

    try:
        # الحصول على الإجابة من Gemini
        response = gemini_model.generate_content(prompt)

        # 🚨 الأجزاء المعدلة لطباعة نظيفة:
        print("\n**الإجابة:**")
        print(response.text)

        # 📚 الكتب المقترحة
        print("\n--- الكتب المقترحة ---")

        for i, idx in enumerate(indices[0]):
            book = df.iloc[idx]
            print(f"{i+1}. {book['Title']} - {book['Author']}")
        # --------------------------------------------------

    except Exception as e:
        print(f"❌ خطأ في Gemini: {e}")
        print("📚 عرض النتائج مباشرة (فشل التوليد):")
        for i, idx in enumerate(indices[0]):
            book = df.iloc[idx]
            print(f"\n{i+1}. 📖 {book['Title']}")
            print(f"   ✍️ {book['Author']}")
            print(f"   📝 {book['Description'][:200]}...")

# ==========================================
# 🧪 تجربة (إزالة الأمثلة الأولية)
# ==========================================
print("\n\n✅ النظام جاهز!")

# وضع تفاعلي
print("\n\n💬 اسألني أي سؤال (أو 'خروج' للإنهاء):")
while True:
    # 🚨 إزالة الـ '\n' من سطر الإدخال لتجنب تكرار سطر فارغ
    q = input("❓ سؤالك: ").strip()
    if q.lower() in ['خروج', 'exit']:
        break
    if q:
        ask_gemini(q)